# Import

In [76]:
import numpy as np
import json

# Classes

In [77]:
# input
class InputLayer:
    def __init__(self, shape, kernel, padding, stride, nb_filter, fct_activation):
        self.shape = shape
        self.kernel = kernel
        self.padding = padding
        self.stride = stride
        self.nb_filter = nb_filter
        self.fct_activation = fct_activation

In [78]:
# Output
class Output:
    def __init__(self, nb_neurones, fct_activation):
        self.nb_neurones = nb_neurones
        self.fct_activation = fct_activation

In [79]:
# Convolution
class Convolution:
    def __init__(self, kernel, padding, stride, nb_filter, fct_activation):
        self.kernel = kernel
        self.padding = padding
        self.stride = stride
        self.nb_filter = nb_filter
        self.fct_activation = fct_activation

In [80]:
# Pooling Avg/Max
class Pooling:
    def __init__(self, op, kernel=2, padding="'valid'", stride=None):
        self.op = op # is the operation wanted (avg/max)
        self.kernel = kernel
        self.padding = padding
        self.stride = stride

In [81]:
# Class Flatten
class Flatten:
    pass

In [82]:
# Dense --> Fully connected layer
class Dense:
    def __init__(self, nb_neurones, fct_activation):
        self.nb_neurones = nb_neurones
        self.fct_activation = fct_activation

# Functions

In [83]:
# function that create CNN with list of layer
def create_CNN(list_layers):
    str_model_cnn = "model = keras.models.Sequential([\n"
    # iterate List of layer
    for layer in list_layers:
        
        # if is Intput
        if isinstance(layer, InputLayer):
            str_model_cnn += "\tkeras.layers.Conv2D({}, kernel_size={}, strides={},  activation={}, input_shape={}, padding={}),\n".format(
                    layer.nb_filter,
                    layer.kernel,
                    layer.stride,
                    layer.fct_activation,
                    layer.shape,
                    layer.padding)
            
        # if is Output
        elif isinstance(layer, Output):
            str_model_cnn += "\tkeras.layers.Dense({}, activation={})\n".format(
                layer.nb_neurones, 
                layer.fct_activation)
            
        # if is Convolution
        elif isinstance(layer, Convolution):
            str_model_cnn += "\tkeras.layers.Conv2D({}, kernel_size={}, strides={}, activation={}, padding={}),\n".format(
                    layer.nb_filter,
                    layer.kernel,
                    layer.stride,
                    layer.fct_activation,
                    layer.padding)
            
        # if is Pooling
        elif isinstance(layer, Pooling):
            if(layer.op == "avg"): # avg Pooling 
                str_model_cnn += "\tkeras.layers.AveragePooling2D(pool_size={}, strides={}, padding={}),\n".format(
                    layer.kernel,
                    layer.stride,
                    layer.padding)
                
            else : # Max Pooling
                str_model_cnn += "\tkeras.layers.MaxPooling2D(pool_size={}, strides={}, padding={}),\n".format(
                    layer.kernel,
                    layer.stride,
                    layer.padding)
                
        # if is Dense (aka Fully connected layer)
        elif isinstance(layer, Dense):
            str_model_cnn += "\tkeras.layers.Dense({}, activation={}),\n".format(
                layer.nb_neurones, 
                layer.fct_activation)
            
        # if is flatten
        elif isinstance(layer, Flatten):
            str_model_cnn += "\tkeras.layers.Flatten(),\n"
            
        # Not possible
        else : print("Not Possible")

    # end model 
    str_model_cnn += "\n])\n"
    return str_model_cnn

In [94]:
# read json file 
# return list of layer
def getArchitectureFromJsonFile(json_file):
    
    # get architecture by json file
    with open(json_file) as archi_json:
        archi = json.load(archi_json)

    # instantiate class
    list_layer = []
    for layer in archi:
        parameters = layer['parameters'] # get parameters
        # if is Input class
        if(layer['class'] == "Input"):
            # instantiate class and add into the list
            list_layer.append(InputLayer(parameters['shape'], 
                                parameters['kernel'], 
                                parameters['padding'], 
                                parameters['stride'],
                                parameters['nb_filter'],
                                parameters['fct_activation']))

        
        # if is Pooling class
        elif(layer['class'] == "Pooling"):
            list_layer.append(Pooling(parameters['op'],
                                      parameters['kernel'],
                                      parameters['padding'],
                                      parameters['stride']))
            
        # if is Convolution class
        elif(layer["class"] == "Convolution"):
            list_layer.append(Convolution(parameters['kernel'],
                                          parameters['padding'],
                                          parameters['stride'],
                                          parameters['nb_filter'],
                                          parameters['fct_activation']))
            
        # if is Flatten class
        elif(layer["class"] == "Flatten"):
            list_layer.append(Flatten())
        
        # if is Dense class
        elif(layer["class"] == "Dense"):
            list_layer.append(Dense(parameters['nb_neurones'],
                                    parameters['fct_activation']))
        # if is Output class
        elif(layer["class"] == "Output"):
            list_layer.append(Dense(parameters['nb_neurones'],
                                    parameters['fct_activation']))
        else : print("Error")
    return list_layer

# Creation CNN valide

In [104]:
# get architecture 
architecture_valid = getArchitectureFromJsonFile("architecture_valid.json")

In [99]:
# reset file
file_ccn_V = open("archi_cnn.py", "w")
file_ccn_V.close()

file_ccn_V = open("archi_cnn.py", "a") # Open file in writting (a --> append)

In [100]:
# write import
file_ccn_V.write("import keras\nimport numpy as np\nfrom keras.utils import plot_model\n")

67

In [101]:
# write model
file_ccn_V.write(create_CNN(architecture_valid))

547

In [102]:
# write the end of file (create png of the model)
file_ccn_V.write("plot_model(model, show_shapes=True, to_file='model_V.png')")

58

In [103]:
# close file
file_ccn_V.close()

# Creation CNN non valide

In [105]:
# get architecture 
architecture_invalid = getArchitectureFromJsonFile("architecture_invalid.json")

In [106]:
# reset file
file_cnn_NV = open("archi_cnn_invalid.py", "w")
file_cnn_NV.close()

file_cnn_NV = open("archi_cnn_invalid.py", "a") # Open file in writting (a --> append)

In [107]:
# write import
file_cnn_NV.write("import keras\nimport numpy as np\nfrom keras.utils import plot_model\n")

67

In [108]:
# write model
file_cnn_NV.write(create_CNN(architecture_invalid))

502

In [109]:
# write the end of file (create png of the model)
file_cnn_NV.write("plot_model(model, show_shapes=True, to_file='model_NV.png')")

59

In [110]:
# close file
file_cnn_NV.close()